importo i pacchetti necessari

In [30]:
import pandas as pd
import requests
import coordinates_calcolator as cc
import url_contructor as uc
import image_visualizator as iv
from db_manager import create_empty_csv, populate_csv, populate_tiles_csv
import toml

prendo i dati dal csv filtrato

In [31]:
mars_data = pd.read_csv('Mars_crater_db_filtered.csv')
zoom = 6
#creo database per le immagini dei singoli crateri, non più in uso
#create_empty_csv('Mars_crater_db_images.csv', ['id', 'image'])

toml_data = toml.load('secrets.toml')
api_key = toml_data['api_key']

abbiamo necessità di gestire il retrieve delle immagini in 8 iterazioni che coinvolgono porzioni diverse della superficie. passiamo alla funzione il numero dell'iterazione corrente (un numero tra 1 e 8) e la funzione restituisce le immagini della porzione interessata, collocandole in data

In [32]:
def get_data(iter, api_key):
    data = []
    
    start=16+(iter-1)*4
    finish=16+iter*4
    for i in range(start, finish):
        for j in range(0, 128):
            url = uc.get_request_url(i, j, tileMatrix=zoom)
            print(url)
            params = {'api_key': api_key}  # Aggiungi la chiave API come parametro
        
            # Esegui la richiesta HTTP con la chiave API
            response = requests.get(url, params=params)
       
            if response.status_code == 200:
                data.append([i, j, response.content])
            else:
                print('error')
    return data

In [38]:
data = get_data(3, api_key)
#da 1 a 8
#fatte 1,2,3


https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/0.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/1.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/2.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/3.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/4.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/5.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/6.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/7.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/8.png
https://trek.nasa.gov/tiles/Mars/EQ/msss_atlas_simp_clon/1.0.0//default/default028mm/6/24/9.png
https://trek.nasa.gov/tiles/Mars/EQ/msss


for i in range(len(data)):
    id = data[i][0]
    print(id)
    image = data[i][1]
    iv.image_visualizator_from_bin(image)


In [39]:
import os
import requests
from PIL import Image
from io import BytesIO

def save_images(data, folder_path):
    # Crea la cartella se non esiste
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for i, j, image_data in data:
        image = Image.open(BytesIO(image_data))
        if f'image_{i}_{j}.png' not in os.listdir(folder_path):
            image_path = os.path.join(folder_path, f'image_{i}_{j}.png')
            image.save(image_path)

In [41]:
save_images(data, 'tante_immagini')

popolo il csv 'Mars_crater_db_images.csv' con i dati ottenuti

In [10]:
#populate_csv(data)

#creo database per le immagini dei tiles che compongono la matrice della porzione di superficie di interesse
#create_empty_csv('Tiles_images_db.csv', ['tile_row','tile_col','image'])
populate_tiles_csv(data)


Added 512 new entries. Total entries: 4096 (before: 3584)


In [11]:
print(len(data))

512


In [7]:
def build_mars_url(x, y, z, p, d='', locale='', bbox='', extent='', sfz=''):
    base_url = "https://trek.nasa.gov/mars/"
    params = {
        'v': '0.1',
        'x': x,
        'y': y,
        'z': z,
        'p': p,
        'd': d,
        'locale': locale,
        'b': bbox,
        'e': extent,
        'sfz': sfz,
        'w': ''  # Il parametro 'w' è vuoto nell'URL originale
    }
    # Costruisci l'URL aggiungendo i parametri come stringa di query
    url = base_url + '?' + '&'.join([f"{key}={value}" for key, value in params.items()])
    return url

In [8]:
def get_mars_data(x, y, z, p, d='', locale='', bbox='', extent='', sfz=''):
    # Costruisci l'URL utilizzando la funzione build_mars_url
    url = build_mars_url(x, y, z, p, d, locale, bbox, extent, sfz)
    print(url)
    try:
        # Esegui la richiesta GET all'URL
        response = requests.get(url)
        # Verifica se la richiesta è andata a buon fine (status code 200)
        if response.status_code == 200:
            # Restituisci il contenuto della risposta
            return response.content
        else:
            # Se la richiesta non è andata a buon fine, stampa un messaggio di errore
            print(f"Errore durante la richiesta GET: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        # Se si verifica un'eccezione durante la richiesta, stampa l'errore
        print(f"Errore durante la richiesta GET: {e}")
        return None

In [9]:
def get_data_from_coordinates(lat,lon,zoom, api_key):
    data = []
    lat, lon = cc.lat_lon_to_tile(lat, lon, zoom)
    url = uc.get_request_url(lat, lon, tileMatrix=zoom)
    print(url)
    params = {'api_key': api_key}  # Aggiungi la chiave API come parametro
    
    # Esegui la richiesta HTTP con la chiave API
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data.append([id,response.content])
    else:
        print('error')
    return data

In [10]:
#data_new = get_data_from_coordinates(27.5,-69.7,2, api_key)
x = 27.5
y = -69.7
z = 2
p = "urn%3Aogc%3Adef%3Acrs%3AEPSG%3A%3A104905"
data_new = get_mars_data(27.5,-69.7,2, api_key)

https://trek.nasa.gov/mars/?v=0.1&x=27.5&y=-69.7&z=2&p=4jSaeeKaa3no2clLYlFzb0IOsGrh50gA8lrTaWIj&d=&locale=&b=&e=&sfz=&w=
